In [1]:
import pandas as pd

df = pd.read_csv("../data/PromoterSet.tsv", sep="\t")
print(df.shape)
print(df.columns.tolist())

(4052, 12)
['pmId', 'pmName', 'strand', 'posTSS', 'sigmaFactor', 'pmSequence', 'firstGeneName', 'distToFirstGene', 'pmEvidence', 'addEvidence', 'confidenceLevel', 'pmids ']


In [2]:
df_tu = pd.read_csv("../data/TUSet.tsv", sep="\t")
print(df_tu.shape)
print(df_tu.columns.tolist())

(3742, 8)
['tuId', 'tuName', 'operonName', 'tuGenes', 'pmName', 'tuEvidence', 'addEvidence', 'confidenceLevel   pmids ']


In [3]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.tools import PythonAstREPLTool
from sqlalchemy import create_engine

engine = create_engine("sqlite:///regulon.db")
df.to_sql("promotores", engine, index=False, if_exists="replace")
df_tu.to_sql("transcription_units", engine, index=False, if_exists="replace")

3742

In [4]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT count(*) FROM promotores WHERE sigmaFactor = 'sigma54';"))
print(db.run("SELECT * FROM transcription_units WHERE tuName = 'yabI';"))

sqlite
['promotores', 'transcription_units']
[(101,)]
[('RDBECOLITUC00096', 'yabI', 'yabI', 'yabI;', 'yabIp', '[COMP-AINF:W]', None, 'W')]


In [5]:
print(db.run("SELECT name FROM sqlite_master WHERE type='table';"))

[('promotores',), ('transcription_units',)]


In [9]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [11]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [20]:
agent_executor.invoke({"input": "Devuelve todos los nombres de los promotores regulados por el sigma factor sigma54 sin límite."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


promotores
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	"pmids " TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids 
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	S	14529615;16861804;9068658 
RDBECOLIPMC00002	yfeKp	forward	2537315.0	sigma24	ccgatgatcctcatcgtaatccaaccgaaactttacctgattctggcagtcaaatcggctAtcacaaaacaaggataaggt	yfeK	-27

{'input': 'Devuelve todos los nombres de los promotores regulados por el sigma factor sigma54 sin límite.',
 'output': 'Los nombres de los promotores regulados por el sigma factor sigma54 son:\n\n1. yahEp\n2. yaaUp\n3. crlp2\n4. nikAp2\n5. emrDp\n6. yhjCp\n7. ygfKp\n8. radDp\n9. htpGp3\n10. flhDp2\n11. actPp\n12. rhaDp\n13. xdhAp2\n14. acrDp2\n15. astCp2\n16. pspAp\n17. potFp1\n18. argTp\n19. prpBp\n20. yeaGp1\n21. yhdWp\n22. ddpXp1\n23. gltIp2\n24. rutAp\n25. htrEp1\n26. rayTp1\n27. ffsp5\n28. ybcKp1\n29. insHp6\n30. ybiOp5\n31. efeOp2\n32. pinEp3\n33. ycgGp7\n34. ycgNp1\n35. pgpBp9\n36. ycjDp5\n37. tusp2\n38. rydBp1\n39. pabBp6\n40. pabBp7\n41. yebVp2\n42. yeiBp2\n43. yfeAp6\n44. tmcAp5\n45. ppkp6\n46. iscXp3\n47. ispFp3\n48. yqeGp5\n49. yggMp1\n50. rfaEp1\n51. yrdBp1\n52. yrdBp4\n53. yheVp1\n54. hslOp4\n55. rsmDp2\n56. yhhMp1\n57. yhiDp1\n58. typAp1\n59. yihLp1\n60. yihLp5\n61. ompLp1\n62. zapBp1\n63. yjaGp4\n64. yjaAp3\n65. ytfFp6\n66. yjhUp1\n67. mcrCp5\n68. rsmCp1\n69. yjjXp6\n70

In [12]:
agent_executor.invoke({"input": "cual es el nombre del transcription unit con el id RDBECOLITUC00081?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


promotores, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	"pmids " TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids 
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	S	14529615;16861804;9068658 
RDBECOLIPMC00002	yfeKp	forward	2537315.0	sigma24	ccgatgatcctcatcgtaatccaaccgaaactttacctgattctggcagtcaaatcggctAtcacaaaa

{'input': 'cual es el nombre del transcription unit con el id RDBECOLITUC00081?',
 'output': 'El nombre del transcription unit con el ID RDBECOLITUC00081 es "yaaX".'}

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

In [17]:
df_1 = pd.read_csv("../data/PromoterSet.tsv", sep="\t")
df_2 = pd.read_csv("../data/TUSet.tsv", sep="\t")
tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """\`\`\`python
{df_name}.head().to_markdown()
>>> {df_head}
\`\`\`"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}"), ])

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "cuales son los nombres de los promotores que esten regulados por el sigma factor sigma54?"
    }
)

323      yahEp
324      yaaUp
325      crlp2
326     nikAp2
327      emrDp
         ...  
4002      nacp
4010     glnKp
4019     hyfAp
4026     zraPp
4027     zraSp
Name: pmName, Length: 101, dtype: object

In [18]:
df_1 = pd.read_csv("../data/PromoterSet.tsv", sep="\t")
df_2 = pd.read_csv("../data/TUSet.tsv", sep="\t")
tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """\`\`\`python
{df_name}.head().to_markdown()
>>> {df_head}
\`\`\`"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}"), MessagesPlaceholder("chat_history", optional=True),])

def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tool)
    .assign(tool_output=itemgetter("ai_msg") | parser | tool)
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["tool_output", "response"])
)

In [19]:
chain.invoke(
    {
        "question": "cuales son los nombres de los promotores que esten regulados por el sigma factor sigma54?"
    }
)

{'tool_output': 323      yahEp
 324      yaaUp
 325      crlp2
 326     nikAp2
 327      emrDp
          ...  
 4002      nacp
 4010     glnKp
 4019     hyfAp
 4026     zraPp
 4027     zraSp
 Name: pmName, Length: 101, dtype: object,
 'response': 'Los nombres de los promotores que están regulados por el sigma factor sigma54 son:\n\n- yahEp\n- yaaUp\n- crlp2\n- nikAp2\n- emrDp\n- nacp\n- glnKp\n- hyfAp\n- zraPp\n- zraSp\n\n(Estos son solo algunos ejemplos, hay un total de 101 promotores regulados por sigma54).'}